# Notebook Lecture 9: Frequency Response
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook introduces frequency response analysis methods for linear time-invariant systems, covering Bode plots, polar plots, and their applications in control system design.

Authors:
- Mark Benazet Castells; mbenazet@ethz.ch
- Felix Muller; fmuller@ethz.ch

## Learning Objectives

After completing this material, you should be able to:

### 1. Master Bode Plot Construction
- Recognize why point-by-point frequency analysis is impractical
- Know and apply the rules for sketching Bode plots
- Understand the significance of corner frequencies and asymptotic behavior

### 2. Apply Construction Rules to Basic Elements
- Draw Bode plots for:
  - Static gains
  - Integrators and differentiators
  - First-order systems
  - Second-order systems
  - Non-minimum phase zeros


## Import the packages:
The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output
from IPython.display import display, clear_output
from scipy import signal

# Frequency Response Recap

## Key Concept: System Response to Sinusoidal Inputs

Linear Time-Invariant (LTI) systems have a fundamental property: when excited by a sinusoidal input, their output is also sinusoidal with:
- Same frequency as the input
- Modified amplitude
- Shifted phase

A sinusoidal input can be written as:
$$u(t) = e^{j\omega t} + e^{-j\omega t} = 2\cos(\omega t)$$

For a system with transfer function $G(s)$, the output is:
$$y(t) = G(j\omega)e^{j\omega t} + G(-j\omega)e^{-j\omega t}$$

Using Euler's formula, this simplifies to:
$$y(t) = |G(j\omega)|\cos(\omega t + \angle G(j\omega))$$

### Key Components
1. **Magnitude Response**: $|G(j\omega)|$
   - Ratio of output amplitude to input amplitude
   - How much the system amplifies/attenuates the input

2. **Phase Response**: $\angle G(j\omega)$
   - Time shift between input and output
   - How much the system delays/advances the signal

As you might expect, the magnitude and phase responses are functions of the input frequency $\omega$. The frequency response of a system is the collection of these responses for all possible input frequencies. Therefore, it is very useful to plot the magnitude and phase responses as functions of frequency, as it will give us a good understanding of how the system behaves for different input frequencies and also give us a good insight into the stability of the system.

The first intuitive thinking on how to plot the frequency response of a system is to evaluate the transfer function at different frequencies and plot the magnitude and phase of the transfer function. 

In [ ]:
# Initialize output widget for plotting
output = widgets.Output()

# Global variables to store points
frequencies = []
magnitudes = []
phases = []

def calculate_response(freq, system_type):
    """Calculate magnitude and phase at given frequency"""
    s = 1j * freq
    if system_type == 'First Order':
        G = 1 / (s + 1)
    elif system_type == 'Second Order':
        G = 1 / (s**2 + s + 1)
    else:  # Integrator
        G = 1 / s
    
    magnitude_db = 20 * np.log10(abs(G))
    phase_deg = np.angle(G, deg=True)
    return magnitude_db, phase_deg

def update_plots(frequency, system_type, show_full=False):
    with output:
        clear_output(wait=True)
        
        # Add new point if not already present
        if frequency not in frequencies:
            mag, phase = calculate_response(frequency, system_type)
            frequencies.append(frequency)
            magnitudes.append(mag)
            phases.append(phase)
            
            # Sort points by frequency for proper line connection
            sorted_indices = np.argsort(frequencies)
            frequencies[:] = [frequencies[i] for i in sorted_indices]
            magnitudes[:] = [magnitudes[i] for i in sorted_indices]
            phases[:] = [phases[i] for i in sorted_indices]
        
        # Create figure with side-by-side plots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot existing points with lines connecting them
        if frequencies:
            ax1.semilogx(frequencies, magnitudes, 'r.-', label='Measured Response')
            ax2.semilogx(frequencies, phases, 'r.-')
        
        # Show complete Bode plot if requested
        if show_full:
            w = np.logspace(-1, 2, 1000)
            mag = []
            phase = []
            for f in w:
                m, p = calculate_response(f, system_type)
                mag.append(m)
                phase.append(p)
            
            ax1.semilogx(w, mag, 'b--', alpha=0.5, label='Complete Response')
            ax2.semilogx(w, phase, 'b--', alpha=0.5)
        
        # Configure magnitude plot
        ax1.grid(True)
        ax1.set_ylabel('Magnitude (dB)')
        ax1.set_xlabel('Frequency (rad/s)')
        ax1.set_title('Magnitude Response')
        if frequencies:
            ax1.legend()
        
        # Configure phase plot
        ax2.grid(True)
        ax2.set_ylabel('Phase (degrees)')
        ax2.set_xlabel('Frequency (rad/s)')
        ax2.set_title('Phase Response')
        
        # Set axis limits
        ax1.set_xlim([0.1, 100])
        ax2.set_xlim([0.1, 100])
        ax1.set_ylim([-40, 5])
        ax2.set_ylim([-180, 10])
        
        plt.tight_layout()
        plt.show()

def clear_points(b):
    """Clear all points"""
    global frequencies, magnitudes, phases
    frequencies = []
    magnitudes = []
    phases = []
    update_plots(freq_slider.value, system_selector.value)

def show_full(b):
    """Show the complete Bode plot"""
    update_plots(freq_slider.value, system_selector.value, show_full=True)

# Create widgets
freq_slider = widgets.FloatLogSlider(
    value=1.0,
    base=10,
    min=-1, # 10^-1 = 0.1
    max=2,  # 10^2 = 100
    step=0.1,
    description='Frequency (rad/s):',
    style={'description_width': 'initial'},
    continuous_update=True  # Update while sliding
)

system_selector = widgets.Dropdown(
    options=['First Order', 'Second Order', 'Integrator'],
    value='First Order',
    description='System Type:',
    style={'description_width': 'initial'}
)

clear_button = widgets.Button(description='Clear Points')
show_full_button = widgets.Button(description='Show Full Response')

# Set up button callbacks
clear_button.on_click(clear_points)
show_full_button.on_click(show_full)

# Create layout
controls = widgets.VBox([
    system_selector,
    freq_slider,
    widgets.HBox([clear_button, show_full_button])
])

# Connect the updates to widget changes
interactive_plot = interactive_output(
    update_plots, 
    {'frequency': freq_slider, 'system_type': system_selector}
)

# Display everything
display(widgets.HBox([controls, output]))

# Initial plot
update_plots(freq_slider.value, system_selector.value)